In [109]:
import pandas as pd
import numpy as np
from scipy import stats 
import os
import sqlite3
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
print(os.listdir('.'))
from sklearn.preprocessing import OneHotEncoder

['modified_dataset.csv', 'hospital_readmissions.csv', 'Log_reg_model.ipynb', 'cleaned_dataset_2.csv', 'project_code.ipynb', '.ipynb_checkpoints', 'hospital_readmission.db', 'hospital_readmissions.csv.zip', 'Arjun Sudhalkar - Project Proposal.pdf', 'result_final.csv']


In [110]:
dataframe = pd.read_csv('./hospital_readmissions.csv')

dataframe.describe()

,time_in_hospital,n_lab_procedures,n_procedures,n_medications,n_outpatient,n_inpatient,n_emergency
count,25000.00000,25000.00000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000
mean,4.45332,43.24076,1.352360,16.252400,0.366400,0.615960,0.186600
std,3.00147,19.81862,1.715179,8.060532,1.195478,1.177951,0.885873
min,1.00000,1.00000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,2.00000,31.00000,0.000000,11.000000,0.000000,0.000000,0.000000
50%,4.00000,44.00000,1.000000,15.000000,0.000000,0.000000,0.000000
75%,6.00000,57.00000,2.000000,20.000000,0.000000,1.000000,0.000000
max,14.00000,113.00000,6.000000,79.000000,33.000000,15.000000,64.000000


In [111]:
dataframe.head(20)

,age,time_in_hospital,n_lab_procedures,n_procedures,n_medications,n_outpatient,n_inpatient,n_emergency,medical_specialty,diag_1,diag_2,diag_3,glucose_test,A1Ctest,change,diabetes_med,readmitted
0,[70-80),8,72,1,18,2,0,0,Missing,Circulatory,Respiratory,Other,no,no,no,yes,no
1,[70-80),3,34,2,13,0,0,0,Other,Other,Other,Other,no,no,no,yes,no
2,[50-60),5,45,0,18,0,0,0,Missing,Circulatory,Circulatory,Circulatory,no,no,yes,yes,yes
3,[70-80),2,36,0,12,1,0,0,Missing,Circulatory,Other,Diabetes,no,no,yes,yes,yes
4,[60-70),1,42,0,7,0,0,0,InternalMedicine,Other,Circulatory,Respiratory,no,no,no,yes,no
5,[40-50),2,51,0,10,0,0,0,Missing,Other,Other,Other,no,no,no,no,yes
6,[50-60),4,44,2,21,0,0,0,Missing,Injury,Other,Other,no,normal,yes,yes,no
7,[60-70),1,19,6,16,0,0,1,Other,Circulatory,Other,Other,no,no,no,yes,yes
8,[80-90),4,67,3,13,0,0,0,InternalMedicine,Digestive,Other,Other,no,no,no,no,yes
9,[70-80),8,37,1,18,0,0,0,Family/GeneralPractice,Respiratory,Respiratory,Other,no,no,yes,yes,no


In [112]:
dataframe_cleaned = dataframe.dropna()

dataframe_cleaned.head()

print("Length of original dataframe:" + str(len(dataframe)))
print("Length of cleaned dataframe:" + str(len(dataframe_cleaned)))

Length of original dataframe:25000
Length of cleaned dataframe:25000


In [113]:
#filtering out the outliers based on a sigificance value of 0.05, with the resulting zscore, 1.96
hospital_time_zscore = np.abs(stats.zscore(dataframe_cleaned['time_in_hospital']))
dataframe_cleaned = dataframe_cleaned[hospital_time_zscore < 1.96]
hospital_outliers = 25000-len(dataframe_cleaned)
print("outliers hospital_time:" + str(25000-len(dataframe_cleaned)))


n_lab_procedures_zscore = np.abs(stats.zscore(dataframe_cleaned['n_lab_procedures']))
dataframe_cleaned = dataframe_cleaned[n_lab_procedures_zscore < 1.96]
n_lab_outliers = 25000-len(dataframe_cleaned) - hospital_outliers
print("outliers n_lab_procedures:" + str(25000-len(dataframe_cleaned) - hospital_outliers))


n_procedures_zscore = np.abs(stats.zscore(dataframe_cleaned['n_procedures']))
dataframe_cleaned = dataframe_cleaned[n_procedures_zscore < 1.96]
n_procedures_outliers = 25000 -len(dataframe_cleaned) - hospital_outliers - n_lab_outliers
print("outliers in n_procedures" + str(n_procedures_outliers))

n_medications_zscore = np.abs(stats.zscore(dataframe_cleaned['n_medications']))
dataframe_cleaned = dataframe_cleaned[n_medications_zscore < 1.96]
n_medications_outliers = 25000 -len(dataframe_cleaned) - hospital_outliers - n_lab_outliers - n_procedures_outliers
print("outliers in n_medications" + str(n_medications_outliers))


#inpatient and emergency have columns of zeros 
print(len(dataframe_cleaned))



outliers hospital_time:1392
outliers n_lab_procedures:1651
outliers in n_procedures1674
outliers in n_medications809
19474


In [114]:
dataframe_cleaned.to_csv('cleaned_dataset_2.csv', index=False)

In [115]:
db = 'hospital_readmission.db'
connection = sqlite3.connect(db)

table = 'hospital_readmission_table'
dataframe_cleaned.to_sql(table, connection, if_exists = 'replace')
query = "SELECT * FROM hospital_readmission_table LIMIT 5;"
print(pd.read_sql(query, connection))

   index      age  time_in_hospital  n_lab_procedures  n_procedures  \
0      0  [70-80)                 8                72             1   
1      1  [70-80)                 3                34             2   
2      2  [50-60)                 5                45             0   
3      3  [70-80)                 2                36             0   
4      4  [60-70)                 1                42             0   

   n_medications  n_outpatient  n_inpatient  n_emergency medical_specialty  \
0             18             2            0            0           Missing   
1             13             0            0            0             Other   
2             18             0            0            0           Missing   
3             12             1            0            0           Missing   
4              7             0            0            0  InternalMedicine   

        diag_1       diag_2       diag_3 glucose_test A1Ctest change  \
0  Circulatory  Respiratory     

In [ ]:
import matplotlib.pyplot as plt
query_medications = "SELECT n_medications FROM hospital_readmission_table"
medications_data = pd.read_sql_query(query_medications, connection)
query_readmitted = "SELECT readmitted FROM hospital_readmission_table"
readmitted_data = pd.read_sql_query(query_readmitted, connection)
readmitted_data['readmitted'] = readmitted_data['readmitted'].map({"yes" :1, "no": 0})

both_query = "SELECT readmitted, n_medications FROM hospital_readmission_table"
both_data = pd.read_sql_query(both_query, connection)
both_data['readmitted'] = both_data['readmitted'].map({"yes":1, "no": 0})
data_cluster = both_data.groupby(['n_medications', 'readmitted']).size().unstack(fill_value = 0)
data_cluster.plot(kind = 'bar', stacked = True, figsize = (12,6))

plt.title('Medications taken vs. Readmission')
plt.xlabel("Medications Taken")
plt.ylabel("Readmission")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

both_query = "SELECT readmitted, n_lab_procedures FROM hospital_readmission_table"
both_data = pd.read_sql_query(both_query, connection)
both_data['readmitted'] = both_data['readmitted'].map({"yes":1, "no": 0})
data_cluster = both_data.groupby(['n_lab_procedures', 'readmitted']).size().unstack(fill_value = 0)
data_cluster.plot(kind = 'bar', stacked = True, figsize = (12,6))

plt.title('Lab Procedures Done vs. Readmission')
plt.xlabel("Lab Procedures Done")
plt.ylabel("Readmission")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [117]:
# remove the missing values, we also want to display how much is missing



age_missing = dataframe_cleaned['age'].isnull().sum()
print(age_missing)
time_in_hospital_missing = dataframe_cleaned['time_in_hospital'].isnull().sum()
print(time_in_hospital_missing)

0
0


In [118]:


#Step one, we one-hot encoded the age data to become numeric, this is a sample (implemented later)
one_hot_encoder = OneHotEncoder(sparse_output = False)

col = dataframe_cleaned['age'].to_frame()
onehot_encoded = one_hot_encoder.fit_transform(col)

onehot_df = pd.DataFrame(onehot_encoded, columns=one_hot_encoder.get_feature_names_out(['age']))
onehot_df.head()

,age_[40-50),age_[50-60),age_[60-70),age_[70-80),age_[80-90),age_[90-100)
0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0


In [119]:
# result = pd.concat([dataframe_cleaned, onehot_df], axis=1)

# result.drop('age', axis=1, inplace=True)
# result.drop('diag_1', axis=1, inplace=True)
# result.drop('diag_2', axis=1, inplace=True)
# result.drop('diag_3', axis=1, inplace=True)
# result.drop('medical_specialty', axis=1, inplace=True)
# result.head()

# result.to_csv('result_final.csv', index=False)


In [120]:
import pandas as pd
import numpy as np
import os

df = pd.read_csv('./hospital_readmissions.csv')



binary_cols = ['change', 'diabetes_med', 'readmitted']
for col in binary_cols:
    df[col] = df[col].map({'no': 0, 'yes': 1})

encoding = {'no': 0, 'normal': 1, 'high': 2}
df['glucose_test'] = df['glucose_test'].map(encoding)
df['A1Ctest'] = df['A1Ctest'].map(encoding)

binary_cols2 = ['diag_1', 'diag_2', 'diag_3']
encoding2 = {'Other': 0, 'Respiratory': 1, 'Circulatory': 2, 'Injury': 3, 'Diabetes': 3}

df['diag_1'] = df['diag_1'].map(encoding2)
df['diag_2'] = df['diag_2'].map(encoding2)
df['diag_3'] = df['diag_3'].map(encoding2)

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['medical_specialty_encoded'] = label_encoder.fit_transform(df['medical_specialty'])


df = df.drop(columns=['medical_specialty'])



def convert_age_to_midpoint(age_range):
    lower, upper = map(int, age_range.strip('[]()').split('-'))
    return (lower + upper) / 2

df['age'] = df['age'].apply(convert_age_to_midpoint)

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

df = df.dropna()
X = df.drop('readmitted', axis=1)  
y = df['readmitted']               


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1] 



coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_[0]
}).sort_values(by='Coefficient', key=abs, ascending=False)


coefficients['Abs_Coefficient'] = coefficients['Coefficient'].abs()
coefficients_sorted = coefficients.sort_values(by='Abs_Coefficient', ascending=False)

print(coefficients_sorted)

                      Feature  Coefficient  Abs_Coefficient
6                 n_inpatient     0.372431         0.372431
14               diabetes_med     0.233094         0.233094
7                 n_emergency     0.217539         0.217539
5                n_outpatient     0.138673         0.138673
8                      diag_1     0.072832         0.072832
3                n_procedures    -0.044261         0.044261
13                     change     0.038963         0.038963
12                    A1Ctest    -0.030419         0.030419
15  medical_specialty_encoded    -0.028036         0.028036
9                      diag_2     0.025749         0.025749
10                     diag_3     0.024240         0.024240
1            time_in_hospital     0.016725         0.016725
11               glucose_test    -0.008318         0.008318
0                         age     0.002753         0.002753
4               n_medications     0.001350         0.001350
2            n_lab_procedures     0.0007

In [122]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score



data = pd.read_csv('./result_final.csv')

categorical_features = ['glucose_test', 'change', 'A1Ctest', 'diabetes_med', 'readmitted']

toEncodeData = data[categorical_features]
encoder = OneHotEncoder(sparse_output = False, drop = 'first')

onehot_encoded = encoder.fit_transform(data[categorical_features])
onehot_encoded_data = pd.DataFrame(onehot_encoded, columns = encoder.get_feature_names_out(categorical_features))
data = pd.concat([data.drop(columns = categorical_features),onehot_encoded_data], axis =1)
data.dropna(inplace = True)

X = data.drop(columns=['readmitted_yes', 'readmitted_nan'])
Y = data['readmitted_yes']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

#classififer implementation and accuracy metrics printed (explained in report)
rf_classifier = RandomForestClassifier(n_estimators =100)
rf_classifier.fit(X_train, Y_train)
Y_pred = rf_classifier.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
report = classification_report(Y_test, Y_pred)
print(report)
print(accuracy)

              precision    recall  f1-score   support

         0.0       0.61      0.62      0.61      1632
         1.0       0.54      0.54      0.54      1395

    accuracy                           0.58      3027
   macro avg       0.58      0.58      0.58      3027
weighted avg       0.58      0.58      0.58      3027

0.5797819623389494
